In [114]:
import pandas as pd
import numpy as np
import tiktoken
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

data = pd.read_csv('../data/spam.csv',encoding='cp437')
print(data.head())
print(data.isna().sum())
print(data.shape)


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64
(5572, 5)


In [ ]:
labels = []
text = []

with open('../data/spam.csv','r') as f :
    lines = f.readlines()
    for i in lines[1:] :
        label = i.split(",")[0]
        labels.append(1 if label == "spam" else 0)
        if i.split(',')[1].startswith('"'):
            text.append(i.split('"')[1])
        else :
            text.append(i.split(',')[1])

print(f"Nombre de label : {len(labels)}")
print(f"Nombre de text : {len(text)}")



Nombre de label : 5574
Nombre de text : 5574


In [116]:
tokenizer = tiktoken.get_encoding("cl100k_base")
train_tokens = [tokenizer.encode(text) for text in text]
print(tokenizer.n_vocab)

100277


In [117]:
seq_lens = [len(seq) for seq in train_tokens]
np.mean(seq_lens)

np.float64(22.21707929673484)

In [118]:
def pad_sequences(sequences, max_length=30):
    return [seq[:max_length] + [0] * (max_length - len(seq)) for seq in sequences]

train_tokens_seq = pad_sequences(train_tokens)
print(len(train_tokens_seq))

5574


In [119]:
class IMDBDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = torch.tensor(texts, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.float32)
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]
    
df_dataset = IMDBDataset(train_tokens_seq, labels)

train_size = int(0.8 * len(df_dataset))
val_size = len(df_dataset) - train_size
train_dataset, val_dataset = random_split(df_dataset, [train_size, val_size])

In [120]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [121]:
text, label = next(iter(train_loader))
print(text)

tensor([[ 1671,   316,   499, 30315,   369, 13985,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [ 5159, 20985,     0,   358,  1390,   499,   311,  1935,   220,    17,
           477,   220,    18,  9364,   315,  6261,  3432,   304, 10107,  3177,
           389,   701,  2849,  4641,     0, 27508,  3177,     0,     0,     0],
        [16384,  1791,  1463,   374,   272,   539,   274,    13,  1472,  1288,
           617,  3309,   757,  6931,    13,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [19701,    11,  4024,   311,  4950,  4216,    11,  3814,  9471,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    9, 15148,  7422,   922, 43560,   25

In [122]:
vocab_size = tokenizer.n_vocab
embedding_layer = nn.Embedding(num_embeddings=vocab_size,
                               embedding_dim=5, 
                               padding_idx=0)
# We create a random list of three integers and use it as input for the embedding
# layer and take a look at the output.
sample_input = torch.tensor([[1, 2, 3]])
embedded_output = embedding_layer(sample_input)
print(embedded_output.shape)  # Output: (3, 5)

torch.Size([1, 3, 5])


In [123]:
embedding_layer.weight.shape

torch.Size([100277, 5])

In [124]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(embed_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)
        pooled = self.pooling(embedded.permute(0, 2, 1)).squeeze(2)
        return torch.sigmoid(self.fc(pooled))

model = TextClassifier(vocab_size=vocab_size,
                      embed_dim=16, 
                      num_class=1)

In [125]:
sample_input.shape

torch.Size([1, 3])

In [126]:
from torchinfo import summary
print(model)

# Print model summary
summary(model, input_data=sample_input)  # (batch_size, input_features)

TextClassifier(
  (embedding): Embedding(100277, 16, padding_idx=0)
  (pooling): AdaptiveAvgPool1d(output_size=1)
  (fc): Linear(in_features=16, out_features=1, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
TextClassifier                           [1, 1]                    --
├─Embedding: 1-1                         [1, 3, 16]                1,604,432
├─AdaptiveAvgPool1d: 1-2                 [1, 16, 1]                --
├─Linear: 1-3                            [1, 1]                    17
Total params: 1,604,449
Trainable params: 1,604,449
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 1.60
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 6.42
Estimated Total Size (MB): 6.42

In [127]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, val_loader, criterion, optimizer, epochs=100):
    """
    Function to train a PyTorch model with training and validation datasets.
    
    Parameters:
    model: The neural network model to train.
    train_loader: DataLoader for the training dataset.
    val_loader: DataLoader for the validation dataset.
    criterion: Loss function (e.g., Binary Cross Entropy for classification).
    optimizer: Optimization algorithm (e.g., Adam, SGD).
    epochs: Number of training epochs (default=100).
    
    Returns:
    history: Dictionary containing loss and accuracy for both training and validation.
    """
    
    # Dictionary to store training & validation loss and accuracy over epochs
    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}
    
    for epoch in range(epochs):  # Loop over the number of epochs
        model.train()  # Set model to training mode
        total_loss, correct = 0, 0  # Initialize total loss and correct predictions
        
        # Training loop
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Reset gradients before each batch
            outputs = model(inputs).squeeze()  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation (compute gradients)
            optimizer.step()  # Update model parameters
            
            total_loss += loss.item()  # Accumulate batch loss
            correct += ((outputs > 0.5) == labels).sum().item()  # Count correct predictions
        
        # Compute average loss and accuracy for training
        train_loss = total_loss / len(train_loader)
        train_acc = correct / len(train_loader.dataset)
        
        # Validation phase (without gradient computation)
        model.eval()  # Set model to evaluation mode
        val_loss, val_correct = 0, 0
        with torch.no_grad():  # No need to compute gradients during validation
            for inputs, labels in val_loader:
                outputs = model(inputs).squeeze()  # Forward pass
                loss = criterion(outputs, labels)  # Compute loss
                val_loss += loss.item()  # Accumulate validation loss
                val_correct += ((outputs > 0.5) == labels).sum().item()  # Count correct predictions
        
        # Compute average loss and accuracy for validation
        val_loss /= len(val_loader)
        val_acc = val_correct / len(val_loader.dataset)
        
        # Store metrics in history dictionary
        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['accuracy'].append(train_acc)
        history['val_accuracy'].append(val_acc)
        
        # Print training progress
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    return history  # Return training history

history = train(model,
                train_loader=train_loader,
                val_loader=val_loader,
                criterion=criterion,
                optimizer=optimizer,
                epochs=10)


Epoch [1/10], Loss: 0.6826, Acc: 0.5355, Val Loss: 0.6269, Val Acc: 0.8188
Epoch [2/10], Loss: 0.5570, Acc: 0.8975, Val Loss: 0.4980, Val Acc: 0.8942
Epoch [3/10], Loss: 0.4279, Acc: 0.9229, Val Loss: 0.3863, Val Acc: 0.9067
Epoch [4/10], Loss: 0.3274, Acc: 0.9368, Val Loss: 0.3080, Val Acc: 0.9211
Epoch [5/10], Loss: 0.2568, Acc: 0.9491, Val Loss: 0.2546, Val Acc: 0.9372
Epoch [6/10], Loss: 0.2087, Acc: 0.9623, Val Loss: 0.2163, Val Acc: 0.9462
Epoch [7/10], Loss: 0.1733, Acc: 0.9708, Val Loss: 0.1883, Val Acc: 0.9552
Epoch [8/10], Loss: 0.1468, Acc: 0.9767, Val Loss: 0.1669, Val Acc: 0.9614
Epoch [9/10], Loss: 0.1266, Acc: 0.9818, Val Loss: 0.1508, Val Acc: 0.9650
Epoch [10/10], Loss: 0.1105, Acc: 0.9843, Val Loss: 0.1378, Val Acc: 0.9713
